<a href="https://colab.research.google.com/github/dvschultz/stylegan2-training/blob/main/SG2_ADA_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#StyleGAN2 Manipulation
#### Inference, Truncation, Interpolation, Mixing, and Projection

This notebook covers manipulating a StyleGAN2-ADA model to produce images and videos. It was made by [Derrick Schultz](https://twitter.com/dvsch) and [Lia Coleman](https://twitter.com/Lialialiacole) for their StyleGAN2 courses.

Parts of this notebook contain ideas and code from [Mikael Christensen](https://colab.research.google.com/drive/1ShgW6wohEFQtqs_znMna3dzrcVoABKIH) and [Dan Shiffman](https://www.youtube.com/watch?v=vEetoBuHj8g).

In [ ]:
%tensorflow_version 1.x
!nvidia-smi

To make demoing easier, we can also add some functions to show images in Colab. Run the next cell to enable that code.

Unfortunately you can’t currently play video inside Colab, so you’ll have to download the videos to view them.

In [ ]:
from IPython.display import Image, display

# Setup: Download the StyleGAN2 repo 

This will install all the necessary libraries to use the StyleGAN2 repo. Press the play button or `shift+return` to run each cell.

Only run the next cell once per session.


In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex

In [ ]:
%cd stylegan2-ada

### Download your trained model from Google Drive
Input the google id of your trained .pkl file. 
The -O option here is to specify the output file name and location.

In [ ]:
!gdown --id 1YzZemZAp7BVW701_BZ7uabJWJJaS2g7v -O /content/network.pkl

#Generate Images

Also known as "Inference", "Evaluation" or "Testing" the model.


In [ ]:
!python generate.py --help

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

In [ ]:
!python generate.py generate-images --network="/content/network.pkl" --seeds=0-10 --outdir="./out/"

Let’s look at a handful of images. 

In [ ]:
listOfImageNames = ['/content/stylegan2-ada/out/seed0000.png',
                    '/content/stylegan2-ada/out/seed0001.png',
                    '/content/stylegan2-ada/out/seed0002.png',
                    '/content/stylegan2-ada/out/seed0003.png',
                    '/content/stylegan2-ada/out/seed0004.png',
                    ]

for imageName in listOfImageNames:
    display(Image(filename=imageName, width=400))

Let’s zip the generated files and download them.

In [ ]:
!zip -r generated-out.zip /content/stylegan2-ada/out

#### Generating Images With A Conditional model
A conditional model is a model trained with class labels. Specify the class you want output with the `--class` option.

In [ ]:
!python generate.py generate-images --network="/content/network.pkl" --seeds=0-10 --outdir="./out/". --class=1

Note: The above command will output it to a directory called `out`. If you already have stuff in `out`, make sure to clear it out. To delete a entire folder in Colab:

In [ ]:
!rm -r "./out/"

# Truncation Traversal


Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically it's infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [ ]:
!python generate.py truncation-traversal --network="/content/network.pkl" --seed=1 --start=-2.0 --stop=2.0 --increment=0.25 --outdir="./tt" --fps=30

There should now be a video called something like: `truncation-traversal-seed1-start-2.0-stop2.0.mp4` in the `tt` folder. You can download just the video by right clicking on it. If you want all the individual frames that are there too, zip the `tt` folder and download it.

In [ ]:
!zip -r generated-tt.zip /content/stylegan2-ada/tt

# Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at two different examples of interpolation: a linear interpolation and a random noise loop.

Both methods require the following options:

`--network`: path to your .pkl file

`--walk-type`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value


### Linear Interpolation

Linear interpolation generates a linear path from one seed to another. The makers of StyleGAN say that doing this in the w space produces the best disentangled interpolations. But let’s start by looking at it in z space.

`--seeds`: Use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop (nice for Instagram and gifs!)

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-z" --seeds=0,1,0 --outdir="./z-walk" --frames=720


Next let’s look at linear interpolation in w space. To do this we set `--walk-type` to `line-w`.

I recommend using the exact same seeds so you see the difference. It’s often very subtle but it is different.


In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-w" --seeds=0,1,0 --outdir="./w-walk" --frames=720

#### Spherical Linear Interpolation (slerp)

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="sphere-z" --seeds=0,1,0 --outdir="./z-sphere" --frames=720

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="sphere-w" --seeds=0,1,0 --outdir="./w-sphere" --frames=720

### Noise Loop Interpolation

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--start_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="noiseloop" --start_seed=0 --outdir="./noise1"

### Circular Loop Interpolation

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="circularloop" --start_seed=0 --diameter=300.0 --frames=720 --outdir="./circleloop1"

# Near Neighbors

**Added by popular demand**

Let’s say you have a seed you like, but want to see other images like it to see if there’s something better. Now you can with the `near-neighbor` argument.

### Options
`--network`, `--seeds`, and `--truncation_psi` work the same as above.

`--diameter`: this sets how far away from the seed you want to generate images. `.0000001` is really close, `.5` is reallly far.

`--num_samples`: how many samples you want to produce

`--save_vector`: this will save the vector as a file in the .npy format. You can then use this for interpolation (not super well supported right now, but can be used manually—see the Projection code for an example of reading a .npy file and interpolating it).




In [ ]:
!python generate.py generate-neighbors --network="/content/network.pkl" --seeds=9 --outdir="./neighbors"

In [ ]:
!zip -r neighbors.zip /content/stylegan2-ada/neighbors

# Style Mixing

Since seeds are just points in a vector space, we can do math things to them, like adding them together. You could do this thru linear interpolation and using the middle frame, but if you want to visualize a number of options here’s a simple script to do it. This takes a number of seeds to produce a grid showing what happens when you add the row and column together (this will make more sense after running it).

In [ ]:
!python style_mixing.py --outdir="./stylemix" --rows=85,100,75,458,1500 --cols=55,821,1789,293 --network="/content/network.pkl"

# Flesh Digressions

[Flesh Digressions](https://aydao.ai/artwork/2020/01/17/fleshdigressions.html) is a technique from [aydao](https://twitter.com/aydaogman). This command will output a flesh digressions video called `circular-25-10-2020-09-00-34-PM.mp4`.

In [ ]:
!python aydao_flesh_digressions.py --pkl "/content/network.pkl" --psi=0.6 --radius_small=10.0 --radius_large=800.00

# Projection

Projection is the process of taking an image from outside the model and finding its nearest representation inside the model. 

###Options 
`--target`: upload an image that you want to project into the model, and then fill the `--target` option with the path to it. This can be any image, as long as it has the same aspect ratio as the dataset that the network was trained on.

`--network`: your model. Just for this example, we're using the FFHQ network in the `--network` option because it's really easy to see what projection is doing when its projection on faces. But of course, you can input your own model as well!

In [ ]:
!python projector.py --outdir="./projection" --target=/content/stylegan2-ada/flower.jpg --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl